# Target-Specific De Novo Peptide Binder Design with DiffPepBuilder

This notebook demonstrates how to use the [DiffPepBuilder](https://github.com/YuzheWangPKU/DiffPepBuilder) package to design peptides that bind to a target protein. We provide an example of the target ALK1 (Activin Receptor-like Kinase 1, PDB ID: [6SF1](https://www.rcsb.org/structure/6SF1)) to demonstrate the procedures of generating peptide binders.

## Setup

In [1]:
#@title ### Download model assets
!git clone https://github.com/YuzheWangPKU/DiffPepBuilder.git
%cd DiffPepBuilder
!tar -xvf SSbuilder/SSBLIB.tar.gz -C SSbuilder
!wget https://zenodo.org/records/12794439/files/diffpepbuilder_v1.pth
!mkdir -p experiments/checkpoints/
!mv diffpepbuilder_v1.pth experiments/checkpoints/
%cd ..

Cloning into 'DiffPepBuilder'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 145 (delta 23), reused 136 (delta 18), pack-reused 0
Receiving objects: 100% (145/145), 4.57 MiB | 6.19 MiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/DiffPepBuilder
tar: unrecognized option '--quiet'
Try 'tar --help' or 'tar --usage' for more information.
--2024-07-22 18:14:51--  https://zenodo.org/records/12794439/files/diffpepbuilder_v1.pth
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1242313672 (1.2G) [application/octet-stream]
Saving to: ‘diffpepbuilder_v1.pth’

diffpepbuilder_v1.p 100%[===================>]   1.16G  12.0MB/s    in 1m 42s  

2024-07-22 18:16:34 (11.6 MB/s) - ‘diffpepbuilder_v1.pth’ saved [1242313672

In [1]:
#@title ### Install dependencies
#@markdown Please restart the runtime as the warning suggests.
!pip install wget wandb fair-esm biotite pyrootutils easydict biopython tqdm ml-collections mdtraj GPUtil dm-tree tmtools
!git clone https://github.com/openmm/pdbfixer.git
%cd pdbfixer
!python setup.py install
%cd ..
!pip install hydra-core hydra-joblib-launcher

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 103.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656

<IPython.core.display.Javascript object>

## Inference

In [2]:
#@title ### Preprocess receptors
%cd DiffPepBuilder
!python experiments/process_receptor.py --pdb_dir examples/receptor_data --write_dir data/receptor_data --peptide_info_path examples/receptor_data/de_novo_cases.json

/content/DiffPepBuilder
Files will be written to data/receptor_data
/content/DiffPepBuilder/experiments/process_receptor.py:278: UserWarning: Both motif and hotspots are found for alk1. The hotspots will be used in priority over the motif.
  warnings.warn(f"Both motif and hotspots are found for {pdb_name}. "
Finished examples/receptor_data/alk1.pdb in 0.12s
Finished processing 1/1 files. Start ESM embedding...
Model file /content/DiffPepBuilder/experiments/checkpoints/esm2_t33_650M_UR50D.pt not found. Downloading...
Model file /content/DiffPepBuilder/experiments/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt not found. Downloading...
Read sequence data with 1 sequences
Processing protein sequence batches:   0% 0/1 [00:00<?, ?it/s]Processing 1 of 1 batches (1 sequences)
Processing protein sequence batches: 100% 1/1 [00:01<00:00,  1.35s/it]
100% 1/1 [00:00<00:00, 89.06it/s]


In [9]:
#@title ### Run *de novo* generation
import os
os.environ['BASE_PATH'] = "/content/DiffPepBuilder"

!torchrun --nproc-per-node=1 experiments/run_inference.py \
  data.val_csv_path=data/receptor_data/metadata_test.csv \
  experiment.use_ddp=False \
  experiment.num_gpus=1 \
  inference.sampling.min_length=12 \
  inference.sampling.max_length=16

/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'inference': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
[2024-07-22 18:40:12,090][experiments.train][INFO] - Loading checkpoint from /content/DiffPepBuilder/experiments/checkpoints/diffpepbuilder_v1.pth
[2024-07-22 18:40:17,977][data.so3_diffuser][INFO] - Computing IGSO3. Saving in /content/DiffPepBuilder/tests/cache/eps_1000_omega_1000_min_sigma_0_1_max_sigma_1_5_schedule_logarithmic
[2024-07-22 18:41:30,611][experiments.train][INFO] - Number of model parameters: 103.66 M
[2024-07-22 18:41:33,156][experiments.train][INFO] - Evaluation mode only, no checkpoint being saved.
[2024-07-22 18:41:33,159][experiments.train][INFO] - Evaluation saved to: /content/DiffPepBuilder/tests/inference_outputs/inference/22D_07M_2024Y_18h_41m
[2024-07-22 18:41:33,294][experiments.train][INFO

In [ ]:
#@title ## Evaluation
#@markdown **Note:** This may take ~15 min
!wget https://downloads.rosettacommons.org/downloads/academic/3.14/rosetta_bin_linux_3.14_bundle.tar.bz2
!tar -xvjf rosetta_bin_linux_3.14_bundle.tar.bz2
!rm -f rosetta_bin_linux_3.14_bundle.tar.bz2
!export ROSETTA_BIN_PATH="rosetta.binary.linux.release-371/main/source/bin"